In [1]:
# Import libraries
import os
import sys
import logging
import pickle # remove
import shutil

current_dir = os.getcwd()
kit_dir = os.path.abspath(os.path.join(current_dir, "..")) # absolute path for ekr_rag directory
repo_dir = os.path.abspath(os.path.join(kit_dir, "../../../")) # absolute path for starter-kit directory
print('kit_dir: %s'%kit_dir)
print('repo_dir: %s'%repo_dir)

sys.path.append(kit_dir)
sys.path.append(repo_dir)

from src.document_retrieval import DocumentRetrieval

CONFIG_PATH = os.path.join(kit_dir,'config.yaml')
PERSIST_DIRECTORY = os.path.join(kit_dir,f"data/my-vector-db")
#save_location = kit_dir + "/data/my-vector-db"

kit_dir: /Users/petrojm/Documents/projects/workshops/ai_engineer_2024/forked/ai-starter-kit/workshops/ai_engineer_2024/ekr_rag
repo_dir: /Users/petrojm/Documents/projects/workshops/ai_engineer_2024/forked/ai-starter-kit


/Users/petrojm/miniconda3/envs/complex_rag/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from PyPDF2 import PdfReader
from langchain_community.document_loaders import UnstructuredPDFLoader

# Specify PDF file
pdf_file = kit_dir + '/data/tmp/2405.07518v1_1page.pdf'

# Initialize DocumentRetrieval class
documentRetrieval =  DocumentRetrieval()
print('Loader: %s'%documentRetrieval.loaders['pdf'])

# Get pdf text
raw_text = []
meta_data = []
if documentRetrieval.loaders['pdf'] == "unstructured":
    loader = UnstructuredPDFLoader(pdf_file)
    docs_unstructured = loader.load()
    for doc in docs_unstructured:
        raw_text.append(doc.page_content)
        meta_data.append({"filename": pdf_file})
elif documentRetrieval.loaders['pdf'] == "pypdf2":
    pdf_reader = PdfReader(pdf_file)
    for page in pdf_reader.pages:
        raw_text.append(page.extract_text())
        meta_data.append({"filename": pdf_file})#, "page": page_number})
else:
    raise ValueError(f"{self.documentRetrieval.loaders['pdf']} is not a valid pdf loader")

# Get the text chunks
text_chunks = documentRetrieval.get_text_chunks_with_metadata(docs=raw_text, meta_data=meta_data) # lst of langchain_core.documents.base.Document
#print(text_chunks[0].page_content)
print('Nb of chunks: %d'%len(text_chunks))


Loader: pypdf2
Nb of chunks: 6


In [3]:
# Create vector store
embeddings = documentRetrieval.load_embedding_model()
if os.path.exists(PERSIST_DIRECTORY):
    shutil.rmtree(PERSIST_DIRECTORY)
    print(f"The directory Chroma has been deleted.")
#vectorstore = documentRetrieval.create_vector_store(text_chunks, embeddings, output_db=None)
vectorstore = documentRetrieval.create_vector_store(text_chunks, embeddings, output_db=PERSIST_DIRECTORY)

# Create conversation chain
documentRetrieval.init_retriever(vectorstore)
conversation = documentRetrieval.get_qa_retrieval_chain()

/Users/petrojm/miniconda3/envs/complex_rag/lib/python3.11/site-packages/threadpoolctl.py:1214: RuntimeWarning: 
Found Intel OpenMP ('libiomp') and LLVM OpenMP ('libomp') loaded at
the same time. Both libraries are known to be incompatible and this
can cause random crashes or deadlocks on Linux when loaded in the
same Python program.
Using threadpoolctl may cause crashes or deadlocks. For more
information and possible workarounds, please see
    https://github.com/joblib/threadpoolctl/blob/master/multiple_openmp.md

  warnings.warn(msg, RuntimeWarning)
2024-06-21 11:03:48,539 [INFO] - Load pretrained SentenceTransformer: intfloat/e5-large-v2


load INSTRUCTOR_Transformer


2024-06-21 11:03:50,183 [INFO] - Use pytorch device: cpu


max_seq_length  512
The directory Chroma has been deleted.


2024-06-21 11:03:50,754 [INFO] - Anonymized telemetry enabled. See                     https://docs.trychroma.com/telemetry for more information.
2024-06-21 11:03:57,501 [INFO] - Vector store saved to /Users/petrojm/Documents/projects/workshops/ai_engineer_2024/forked/ai-starter-kit/workshops/ai_engineer_2024/ekr_rag/data/my-vector-db


In [4]:
# Ask questions about your data
user_question = "What is a monolithic model?"

response = conversation.invoke({"question":user_question})
print(response['question'])
print(response['answer'])
#print(len(response['source_documents']))

for i in range(0,len(response['source_documents'])):
    print('\nSource #%d:'%(i+1))
    print(response['source_documents'][i].page_content)
    print(response['source_documents'][i].metadata)

2024-06-21 11:04:00,848 [WARNING] - Number of requested results 15 is greater than number of elements in index 6, updating n_results = 6


What is a monolithic model?
A monolithic model is a large, single model that has billions or trillions of parameters, requiring significant computational resources and expertise to train and serve.

Source #1:
Source: /Users/petrojm/Documents/projects/workshops/ai_engineer_2024/forked/ai-starter-kit/workshops/ai_engineer_2024/ekr_rag/data/tmp/2405, Text: 
and updated on their data [ 1], [10], [16]. Finally, systems
that cater to monolothic models have scaled compute TFLOPs
much faster than memory bandwidth and capacity, creating the
memory wall [ 35] where the memory system can no longer
feed the compute efﬁciently.
The ML research community has responded with
ecosystems of much smaller, modular models that are
just as capable, but are cheaper and easier to train and
serve [ 5], [37], [43], [58]. Smaller models like the 7B-
parameter Llama 3 [ 13], Llama 2 [ 68], and Mistral 7B [ 44]
are often adequate. They might not match the performance
of larger models over a general suite of tasks